This is a markdown cell.

In [1]:
println("This is a Julia cell")

This is a Julia cell


In [2]:
using DFTK
using Plots

**Goal**: Optimize the lattice parameter of graphene!

1. give a lattice parameter
2. calculate the energy
3. save that value
4. repeat until we find the smallest energy

In [3]:
function computeEnergy(a)
    # lattice parameters
    a = a/0.53   # convert units from angstroms to bohr
    c = 10/0.53
    
    # building the crystal in the language that DFTK wants
    C = ElementPsp(:C, psp=load_psp(:C, functional="lda"))
    positions = [[1/3., 2/3., 0], [2/3., 1/3., 0]]
    atoms = [C => positions]
    lattice = [[a -0.5*a 0]; 
               [0 0.5*sqrt(3)*a 0]; 
               [0 0 c]]
    
    # building a model to calculate energy
    model = model_LDA(lattice, atoms, temperature=0.01)
    basis = PlaneWaveBasis(model, 15, kgrid=[20, 20, 2])
    scfres = self_consistent_field(basis; callback=info->nothing) # calculates energy!
    E = scfres.energies.total;
    return E
end;

In [5]:
function calculateEnergies(trial_a)
    
    # trial_a : list of lattice parameters to compute the energy for
    # return : list of energies corresponding to trial_a
    
    energies = []
    # for each a compute the energy
    for a in trial_a
        E = computeEnergy(a)
        append!(energies, E)
        println("$a      $E")
    end
    return energies
end

calculateEnergies (generic function with 1 method)

In [20]:
trial_a = collect(2:0.1:3)
@time energies = calculateEnergies(trial_a);

2.0      -10.825827348904541
2.1      -10.987122874881104
2.2      -11.091613826226467
2.3      -11.152555430516843
2.4      -11.18115718957586
2.5      -11.186239240576642
2.6      -11.17587875520656
2.7      -11.156015185805485
2.8      -11.129825069568785
2.9      -11.098942909194125
3.0      -11.064674442386458
345.205670 seconds (30.73 M allocations: 60.655 GiB, 3.04% gc time)


In [21]:
hcat([trial_a, energies]...);

In [22]:
plot(trial_a, energies, legend=false, m=:circle, markersize=3, linestyle=:solid)
plot!(title = "Graphene Optimization", ylabel= "Energy (Ha)", xlabel = "a (ang.)")
annotate!(2.5, -10.9, "a ≈ 2.5 ang")
savefig("graphene_optim.pdf")